In [2]:
from functools import reduce
import numpy as np
import pandas as pd

# Import, check and prepare data

Import CSV files that contain:
- The PIDs of all datasets published by many repositories that use the Dataverse software and which repositories published them
- Author field metadata entered in all of those datasets

In [3]:
# Get basic info about datasets in repositories except for ODISSEI Portal
datasetPIDsDF = (pd
    .read_csv(
        'dataset_pids_from_most_known_dataverse_installations_2023.08.csv',
        sep=',',
        na_filter=False)
    .query(
        '(dataverse_json_export_saved == True) and\
        (dataverse_installation_name != "ODISSEI_Portal")')
    .drop(columns=['dataverse_json_export_saved'])
    .reset_index(drop=True, inplace=False)
 )

datasetPIDsDF.head()

,dataverse_installation_name,dataset_pid_url,dataverse_collection_alias,dataverse_collection_name,dataverse_collection_type
0,UNB_Libraries_Dataverse,https://doi.org/10.25545/NVN79Z,snb,Service New Brunswick,ORGANIZATIONS_INSTITUTIONS
1,UNB_Libraries_Dataverse,https://doi.org/10.25545/YB60PU,blightchemistry,Blight Chemistry Research Omniverse,RESEARCH_GROUP
2,UNB_Libraries_Dataverse,https://doi.org/10.25545/WHD3KC,blightchemistry,Blight Chemistry Research Omniverse,RESEARCH_GROUP
3,UNB_Libraries_Dataverse,https://doi.org/10.25545/6TZWCG,Ir-CationBinding,Elucidation of Charge Contributions in Iridium-Chelated Hydrogen-Bonding Systems,JOURNALS
4,UNB_Libraries_Dataverse,https://doi.org/10.25545/NYM13B,IFMLAB,Integrated Forest Management Lab,RESEARCH_GROUP


In [4]:
# Get Author field metadata entered in all datasets in Dataverse repositories
# and remove datasets have have no author metadata
authorMetadataDF = (pd
    .read_csv(
        'author(citation)_2023.08.22-2023.08.28.csv',
        sep=',',
        na_filter=True)
    .drop(columns=['dataset_pid', 'authorAffiliation'])
    .query('authorName != "N/A"')
    .reset_index(drop=True, inplace=False)
    )

authorMetadataDF.head()

,dataset_pid_url,dataset_publication_date,dataset_version_number,dataset_version_create_time,authorName,authorIdentifierScheme,authorIdentifier
0,https://doi.org/10.48370/OFD/8TTRLC,2022-05-13,1.0,2022-05-11T14:44:29Z,"Mammal Research Institute, Polish Academy of Sciences",NaN,NaN
1,https://doi.org/10.17026/dans-26a-cq4r,2015-09-18,2.0,2022-02-18T19:29:27Z,H.J. Hesseling,NaN,NaN
2,https://doi.org/10.17026/dans-zc2-mc2g,2016-12-31,1.0,2022-02-24T21:01:03Z,S. Moerman,NaN,NaN
3,https://doi.org/10.17026/dans-x9z-bmn6,2020-12-22,1.0,2022-02-14T05:11:03Z,G. Zielman,NaN,NaN
4,https://doi.org/10.7910/DVN/QD0V0H,2021-09-20,1.0,2021-07-29T19:45:43Z,"Master, Daniel M.",NaN,NaN


In [5]:
print(f'Number of datasets in datasetPIDsDF: {len(datasetPIDsDF)}')
datasetCountInAuthorMetadataDF = len(pd.unique(authorMetadataDF['dataset_pid_url']))
print(f'Number of datasets in authorMetadataDF: {datasetCountInAuthorMetadataDF}')

Number of datasets in datasetPIDsDF: 390401
Number of datasets in authorMetadataDF: 390401


In [6]:
# Join the datasetPIDsDF and the authorMetadataDF to add the installation column,
# so we know which installations published each dataset
# and remove datasets that have no author
datasetPIDsAndAuthorMetadataDF = (pd
    .merge(datasetPIDsDF, authorMetadataDF,
        how='inner',
        on=['dataset_pid_url'])
    .drop(columns=['dataset_version_number'])
    .reset_index(drop=True, inplace=False))

# Make sure the count of rows in datasetPIDsAndAuthorMetadataDF
# is the same as the count of total datasets in datasetPIDsDF: 390401
datasetCountInDatasetPIDsAndAuthorMetadataDF = len(pd.unique(datasetPIDsAndAuthorMetadataDF['dataset_pid_url']))
print(f'Number of datasets in datasetPIDsAndAuthorMetadataDF: {datasetCountInDatasetPIDsAndAuthorMetadataDF}')

print(f'Number of author metadata in datasetPIDsAndAuthorMetadataDF: {len(datasetPIDsAndAuthorMetadataDF)}')

countOfInstallations = len(datasetPIDsAndAuthorMetadataDF.dataverse_installation_name.unique())
print(f'Number of installations in datasetPIDsAndAuthorMetadataDF: {countOfInstallations}')

Number of datasets in datasetPIDsAndAuthorMetadataDF: 390401
Number of author metadata in datasetPIDsAndAuthorMetadataDF: 667435
Number of installations in datasetPIDsAndAuthorMetadataDF: 84


In [7]:
datasetPIDsAndAuthorMetadataDF.head()

,dataverse_installation_name,dataset_pid_url,dataverse_collection_alias,dataverse_collection_name,dataverse_collection_type,dataset_publication_date,dataset_version_create_time,authorName,authorIdentifierScheme,authorIdentifier
0,UNB_Libraries_Dataverse,https://doi.org/10.25545/NVN79Z,snb,Service New Brunswick,ORGANIZATIONS_INSTITUTIONS,2018-05-10,2022-03-23T19:49:10Z,Service New Brunswick,NaN,NaN
1,UNB_Libraries_Dataverse,https://doi.org/10.25545/YB60PU,blightchemistry,Blight Chemistry Research Omniverse,RESEARCH_GROUP,2018-06-20,2018-02-21T18:26:43Z,"Blight, Barry A",ORCID,0000-0003-1166-6206
2,UNB_Libraries_Dataverse,https://doi.org/10.25545/WHD3KC,blightchemistry,Blight Chemistry Research Omniverse,RESEARCH_GROUP,2019-06-23,2019-06-21T18:30:39Z,"Balonova, Barbora",NaN,NaN
3,UNB_Libraries_Dataverse,https://doi.org/10.25545/6TZWCG,Ir-CationBinding,Elucidation of Charge Contributions in Iridium-Chelated Hydrogen-Bonding Systems,JOURNALS,2021-05-18,2021-05-18T16:39:24Z,"Blight, Barry A",NaN,NaN
4,UNB_Libraries_Dataverse,https://doi.org/10.25545/NYM13B,IFMLAB,Integrated Forest Management Lab,RESEARCH_GROUP,2020-06-30,2020-11-30T22:44:12Z,"Chen, Yingbing",NaN,NaN


# Explore data

In [11]:
# Get info about author metadata that includes no identifier metadata,
# partial identifier metadata, and complete identifier metadata
authorsWithNoIdentifiersDf = (
    datasetPIDsAndAuthorMetadataDF
        .query(
            'authorIdentifierScheme != authorIdentifierScheme and\
            authorIdentifier != authorIdentifier')
        .reset_index(drop=True, inplace=False)
)

authorsWithPartialIdentifiersDf = (
    datasetPIDsAndAuthorMetadataDF
        .query(
            '(authorIdentifierScheme == authorIdentifierScheme and\
             authorIdentifier != authorIdentifier) or\
            (authorIdentifierScheme != authorIdentifierScheme and\
             authorIdentifier == authorIdentifier)')
        .fillna('')
        .reset_index(drop=True, inplace=False)
)

authorsWithCompleteIdentifiersDf = (
    datasetPIDsAndAuthorMetadataDF
        .query(
            'authorIdentifierScheme == authorIdentifierScheme and\
             authorIdentifier == authorIdentifier')
        .reset_index(drop=True, inplace=False)
)

sumOfAuthorMetadata = (
    len(authorsWithPartialIdentifiersDf)
    + len(authorsWithCompleteIdentifiersDf)
    + len(authorsWithNoIdentifiersDf)
)
print(f'Number of author metadata with partial, complete, and no identifier metadata: {sumOfAuthorMetadata}')

print(f'Number of author metadata with partial identifier metadata: {len(authorsWithPartialIdentifiersDf)}')
print(f'Number of author metadata with complete identifier metadata: {len(authorsWithCompleteIdentifiersDf)}')
print(f'Number of author metadata with no identifier metadata: {len(authorsWithNoIdentifiersDf)}')

Number of author metadata with partial, complete, and no identifier metadata: 667435
Number of author metadata with partial identifier metadata: 4438
Number of author metadata with complete identifier metadata: 82078
Number of author metadata with no identifier metadata: 580919


Get count of author metadata with and without identifiers for each installation

In [12]:
# Count of author metadata in authorsWithNoIdentifiersDf per installation
authorsWithNoIdentifiersByInstallationDf = (
    authorsWithNoIdentifiersDf
        .value_counts(subset=['dataverse_installation_name'])
        .to_frame('count_of_authors_with_no_identifiers')
)

# Count of author metadata in authorsWithPartialIdentifiersDf per installation
authorsWithPartialIdentifiersByInstallationDf = (
    authorsWithPartialIdentifiersDf
        .value_counts(subset=['dataverse_installation_name'])
        .to_frame('count_of_authors_with_partial_identifiers')
)

# Count of author metadata in authorsWithCompleteIdentifiersDf per installation
authorsWithCompleteIdentifiersByInstallationDf = (
    authorsWithCompleteIdentifiersDf
        .value_counts(subset=['dataverse_installation_name'])
        .to_frame('count_of_authors_with_complete_identifiers')
)

# Join the three dataframes
dataframes = [
    authorsWithNoIdentifiersByInstallationDf,
    authorsWithPartialIdentifiersByInstallationDf,
    authorsWithCompleteIdentifiersByInstallationDf
]

countOfAuthorsByInstallationDf = reduce(lambda left, right: left.join(right, how='outer'), dataframes)
countOfAuthorsByInstallationDf = (
    countOfAuthorsByInstallationDf
        .fillna(0)
        .astype('int32')
        .reset_index(drop=False, inplace=False)
        # Reorder columns
        .loc[:,[
            'dataverse_installation_name',
            'count_of_authors_with_complete_identifiers',
            'count_of_authors_with_partial_identifiers',
            'count_of_authors_with_no_identifiers']]
)

countOfAuthorsByInstallationDf.head()

,dataverse_installation_name,count_of_authors_with_complete_identifiers,count_of_authors_with_partial_identifiers,count_of_authors_with_no_identifiers
0,ACSS_Dataverse,5,0,141
1,ADA_Dataverse,1102,22,2140
2,ASU_Library_Research_Data_Repository,92,4,115
3,AUSSDA_Dataverse,258,0,2104
4,Abacus,0,1,4083


In [13]:
# Get number of author metadata with each type of identifier, e.g. ORCID, GND
countOfValidIdentifiersByTypeAndInstallationDf = (pd
    .crosstab(
        authorsWithCompleteIdentifiersDf.dataverse_installation_name,
        authorsWithCompleteIdentifiersDf.authorIdentifierScheme,
        margins=True, margins_name='Total')
    .reset_index(drop=False, inplace=False)
    .rename_axis(None, axis=1)
)

countOfValidIdentifiersByTypeAndInstallationDf.head()

,dataverse_installation_name,DAI,DOI,GND,ISNI,LCNA,ORCID,ResearcherID,ScopusID,VIAF,idHAL,Total
0,ACSS_Dataverse,0,0,0,0,0,5,0,0,0,0,5
1,ADA_Dataverse,0,0,0,0,0,1102,0,0,0,0,1102
2,ASU_Library_Research_Data_Repository,0,0,0,0,0,92,0,0,0,0,92
3,AUSSDA_Dataverse,0,0,0,0,0,258,0,0,0,0,258
4,Arca_Dados,0,0,0,0,0,132,0,0,0,0,132


In [36]:
# Export the dataframe as a CSV file
countOfValidIdentifiersByTypeAndInstallationDf.to_csv('countOfValidIdentifiersByTypeAndInstallationDf.csv', index=False)

In [43]:
# How much author metadata have ORCIDs but no Identifier Type selected?
authorMetadataWithInvalidORCIDsDf = (
    authorsWithPartialIdentifiersDf
        .query(
            'authorIdentifier.str.contains("orcid", case=False) or\
            authorIdentifier.str.match(".*\d{4}-\d{4}-\d{4}-\d{4}.*")',
            engine='python')
        .reset_index(drop=True, inplace=False)
)

authorMetadataWithInvalidORCIDsDf.head()

,dataverse_installation_name,dataset_pid_url,dataverse_collection_alias,dataverse_collection_name,dataverse_collection_type,dataset_publication_date,dataset_version_create_time,authorName,authorIdentifierScheme,authorIdentifier
0,Harvard_Dataverse,https://doi.org/10.7910/DVN/IVSH2F,BIVES,Virginia’s Barrier Islands under Differing Sea-Level Rise Scenarios Dataverse,RESEARCH_PROJECTS,2015-12-10,2017-10-09T17:53:01Z,"Hamilton, Stuart",,0000-0001-8114-7247
1,Harvard_Dataverse,https://doi.org/10.7910/DVN/FYOEWG,BJPolS,British Journal of Political Science Dataverse,JOURNALS,2021-04-12,2021-01-13T18:08:40Z,"Nathan, Noah L.",,0000-0002-3681-4925
2,Harvard_Dataverse,https://doi.org/10.7910/DVN/DQ1W55,BJPolS,British Journal of Political Science Dataverse,JOURNALS,2019-12-10,2019-08-08T12:33:18Z,Steffen Hertog,,0000-0002-6758-9564
3,Harvard_Dataverse,https://doi.org/10.7910/DVN/Y3QFBP,BJPolS,British Journal of Political Science Dataverse,JOURNALS,2017-07-11,2017-02-27T15:40:08Z,"Brandenberger, Laurence",,0000-0003-0392-9766
4,Harvard_Dataverse,https://doi.org/10.7910/DVN/FEQSC7,BJPolS,British Journal of Political Science Dataverse,JOURNALS,2022-03-14,2021-12-15T11:23:55Z,"Müller, Wolfgang C.",,0000-0001-9085-9465


In [42]:
countOfAuthorsWithInvalidORCIDsByInstallationDf = (
    authorMetadataWithInvalidORCIDsDf
        .value_counts(subset=['dataverse_installation_name'])
        .to_frame('count_of_authors_with_invalid_orcids')
        .reset_index(drop=False, inplace=False)
)

countOfAuthorsWithInvalidORCIDsByInstallationDf.head()

,dataverse_installation_name,count_of_authors_with_invalid_orcids
0,Edmond,766
1,Harvard_Dataverse,234
2,Recherche_Data_Gouv,73
3,CIMMYT_Research_Data,38
4,Borealis,37


In [ ]:
# Lets add this count of "invalid" ORCIDs to the count of "valid" ORCIDs in  countOfValidIdentifiersByTypeAndInstallationDf

In [ ]:
# How often do installations publish datasets with author identifiers
# that aren't ORCIDs
